Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [2]:
!pip install watson-developer-cloud==1.5

Requirement not upgraded as not directly required: watson-developer-cloud==1.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: autobahn>=0.10.9 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: python-dateutil>=2.5.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: pyOpenSSL>=16.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: service-identity>=17.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: requests<3.0,>=2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as n

In [3]:
  from bs4 import BeautifulSoup
  from watson_developer_cloud import NaturalLanguageUnderstandingV1
  from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions, SemanticRolesOptions, SentimentOptions, EmotionOptions, ConceptsOptions, CategoriesOptions

In [4]:
apikey='2lm8vP9V5AAZUuKXsQrqWQL5G-BSAx3JG5uYvf_h2HDu'
url='https://gateway.watsonplatform.net/natural-language-understanding/api'
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-01-25',
    iam_api_key=apikey,
    url=url
)

In [10]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
content = r.text
soup = BeautifulSoup(content, 'lxml')


In [11]:
data_table = soup.find('table', class_='sortable')

In [9]:
#convenrt the table into a datafram
df = pd.read_html(str(data_table))
df_json = df[0].to_json(orient='records')
df_json
df_correct = pd.read_json(df_json)

#set headers
new_header = df_correct.iloc[0] 
df_correct = df_correct[1:] 
df_correct.columns = new_header 

#Clean dataset
df_correct = df_correct[df_correct.Borough != 'Not assigned']

df_correct = df_correct.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_correct.columns = ['Postcode', 'Borough', 'Neighbourhood']

df_correct['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_correct
df_correct.shape




(103, 3)